In [ ]:
import itertools
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm
from dataclasses import asdict, dataclass, field
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd

import vpype_cli
from typing import List, Generic
from genpen import genpen as gp, utils as utils
from scipy import stats as ss
import geopandas
from shapely.errors import TopologicalError
import functools
%load_ext autoreload
%autoreload 2

In [ ]:
page_x_inches: float = 11. # inches
page_y_inches: float = 14 # inches
border:float = 20.

buffer_style = 2

In [ ]:
px = utils.DistanceConverter(page_x_inches, 'inches').mm
py = utils.DistanceConverter(page_y_inches, 'inches').mm
page_format = f'{px}mmx{py}mm'
drawbox = sg.box(border, border, px-border, py-border)

xmin, ymin, xmax, ymax = drawbox.bounds


In [ ]:
planet = Point(140,-150).buffer(250, resolution=80)
planet_in_frame = planet.intersection(drawbox)

In [ ]:
moon = Point(220, 120).buffer(12)

space = drawbox.difference(planet).difference(moon)

space

In [ ]:
# poly = space

# pts = gp.get_random_points_in_polygon(poly, n_points=600)

In [ ]:

# circle_to_fill = space
# # init
# fc_rad = gp.get_rad(circle_to_fill)
# x = circle_to_fill.centroid.x
# rads = np.random.uniform(4, 5, 20)
# circles = gp.circle_pack_within_poly(circle_to_fill, rads, progress_bar=True, max_additions=5)

In [ ]:
(xbins, ybins), (xs, ys) = gp.overlay_grid(space, xstep=2, ystep=2, flatmesh=True)

pts = [Point((x,y) + np.random.randn(2)*1) for x,y in zip(xs, ys)]

In [ ]:
circles = MultiPolygon([p.buffer(1.2) for p in pts])

In [ ]:
stars = []
for c in circles:
    d = c.distance(planet)
    buffer_distance = np.interp(d, [0, 100], [-1.3, -0.8]) + np.random.randn()* 0.1
    bc = c.buffer(buffer_distance)
    stars.append(bc)
    
stars = MultiPolygon(stars)

In [ ]:
negative_space = gp.robust_difference(space, stars)

In [ ]:
fill_curve = planet.boundary.intersection(space)
fill_curve = so.linemerge(fill_curve)

In [ ]:
max_height = drawbox.bounds[3]

In [ ]:
space_delta_yoff_start = 1.2
space_delta_yoff_end = 0.6
atmo_yoff_start = 2.

In [ ]:
atmo_delta_yoffs = np.linspace(atmo_yoff_start, space_delta_yoff_start, 9)

In [ ]:
yoff_deltas = list(atmo_delta_yoffs) + list(np.linspace(space_delta_yoff_start, space_delta_yoff_end,140)) + [space_delta_yoff_end] * 300

In [ ]:
yoffs = np.cumsum(yoff_deltas)

In [ ]:
fills = []
for i, yoff in enumerate(yoffs):
    y_shift_curve = sa.translate(fill_curve, yoff=yoff)
    if y_shift_curve.bounds[1] > max_height:
        break
        
    y_shift_curve = y_shift_curve.intersection(negative_space)
    
    if y_shift_curve.type == 'MultiLineString':
        y_shift_curves = list(y_shift_curve)
    else:
        y_shift_curves = [y_shift_curve]
        
    
    if (i % 2) == 0:
        x_order = np.argsort([-ysc.centroid.x for ysc in y_shift_curves])
        y_shift_curves = [y_shift_curves[i] for i in x_order]
        y_shift_curves = [LineString(np.flipud(np.array(ysc))) for ysc in y_shift_curves]
    fills+= y_shift_curves
    
    


In [ ]:
positive_space = gp.merge_LineStrings(fills)

In [ ]:
positive_space = sa.rotate(positive_space, 180)

In [ ]:
sk = vsketch.Vsketch()
sk.size(page_format)

sk.scale('1mm')
# sk.stroke(1)
# sk.geometry(stars)
# sk.stroke(2)
# sk.geometry(planet_in_frame)
# sk.stroke(3)
# sk.geometry(moon)
sk.geometry(positive_space)
sk.display(color_mode='none')

In [ ]:
savepath = '/mnt/c/code/side/plotter_images/oned_outputs/225_space_is_good.svg'
sk.vpype('linemerge --tolerance 0.1 -f')
sk.save(savepath)

# a little more license

In [ ]:
page_x_inches: float = 11. # inches
page_y_inches: float = 17 # inches
border:float = 20.

buffer_style = 2

In [ ]:
px = utils.DistanceConverter(page_x_inches, 'inches').mm
py = utils.DistanceConverter(page_y_inches, 'inches').mm
page_format = f'{px}mmx{py}mm'
drawbox = sg.box(border, border, px-border, py-border)

xmin, ymin, xmax, ymax = drawbox.bounds


In [ ]:
planet = Point(140,-150).buffer(250, resolution=80)
planet_in_frame = planet.intersection(drawbox)

In [ ]:
moon = Point(210, 120).buffer(15)

space = drawbox.difference(planet).difference(moon)

space

In [ ]:
# poly = space

# pts = gp.get_random_points_in_polygon(poly, n_points=600)

In [ ]:

# circle_to_fill = space
# # init
# fc_rad = gp.get_rad(circle_to_fill)
# x = circle_to_fill.centroid.x
# rads = np.random.uniform(4, 5, 20)
# circles = gp.circle_pack_within_poly(circle_to_fill, rads, progress_bar=True, max_additions=5)

In [ ]:
(xbins, ybins), (xs, ys) = gp.overlay_grid(space, xstep=3., ystep=3., flatmesh=True)

pts = [Point((x,y) + np.random.randn(2)*2) for x,y in zip(xs, ys)]

In [ ]:
circles = []
for p in pts:
    d = p.distance(planet)
    buffer_distance = np.interp(d, [0, 150], [-0.3, 2.5]) + np.interp(d, [350, 540], [0., 2.5])+ np.random.randn()* 0.3
    if d > 0:
        circles.append(p.buffer(buffer_distance))

In [ ]:
stars = MultiPolygon(circles)

In [ ]:
negative_space = gp.robust_difference(space, stars)

In [ ]:
fill_curve = planet.boundary.intersection(space)
fill_curve = so.linemerge(fill_curve)

In [ ]:
max_height = drawbox.bounds[3]

In [ ]:
space_delta_yoff_start = 1.2
space_delta_yoff_end = 0.6
atmo_yoff_start = 2.5

In [ ]:
atmo_delta_yoffs = np.linspace(atmo_yoff_start, space_delta_yoff_start, 8)

In [ ]:
yoff_deltas = list(atmo_delta_yoffs) + list(np.linspace(space_delta_yoff_start, space_delta_yoff_end,140)) + [space_delta_yoff_end] * 300

In [ ]:
yoffs = np.cumsum(yoff_deltas)

In [ ]:
fills = []
for i, yoff in enumerate(yoffs):
    y_shift_curve = sa.translate(fill_curve, yoff=yoff)
    if y_shift_curve.bounds[1] > max_height:
        break
        
    y_shift_curve = y_shift_curve.intersection(negative_space)
    if y_shift_curve.length < 1e-3:
        break
        
    if y_shift_curve.type == 'MultiLineString':
        y_shift_curves = list(y_shift_curve)
    else:
        y_shift_curves = [y_shift_curve]
        
    
    if (i % 2) == 0:
        x_order = np.argsort([-ysc.centroid.x for ysc in y_shift_curves])
        y_shift_curves = [y_shift_curves[i] for i in x_order]
        y_shift_curves = [LineString(np.flipud(np.array(ysc))) for ysc in y_shift_curves]
    fills+= y_shift_curves
    
    


In [ ]:
positive_space = gp.merge_LineStrings(fills)

In [ ]:
positive_space = sa.rotate(positive_space, 180)

In [ ]:
sk = vsketch.Vsketch()
sk.size(page_format)

sk.scale('1mm')
sk.geometry(positive_space)
sk.display(color_mode='none')

In [ ]:
savepath = '/mnt/c/code/side/plotter_images/oned_outputs/222_space_is_good.svg'
sk.vpype('linemerge --tolerance 0.1 -f')
sk.save(savepath)